In [1]:
import copy
import itertools
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import pywt
from IPython.display import clear_output
from pynufft import NUFFT
from scipy import signal as sci_signal
from scipy.constants import c

from csromer.base import Dataset
from csromer.dictionaries.discrete import DiscreteWavelet
from csromer.dictionaries.undecimated import UndecimatedWavelet
from csromer.io import Reader, Writer
from csromer.objectivefunction import L1, TSV, TV, Chi2, OFunction
from csromer.optimization import ADMM, FISTA, SDMM, GradientBasedMethod
from csromer.reconstruction import Parameter
from csromer.simulation import FaradayThickSource, FaradayThinSource
from csromer.transformers import DFT1D, NUFFT1D, Gridding
from csromer.utils import Gaussian, complex_to_real, real_to_complex

%matplotlib inline

#np.random.seed(666)

In [2]:
def gini_coefficient(w):
    # Order vector
    w_ordered = np.sort(np.abs(w), kind="stable")
    l1_norm = np.sum(np.abs(w_ordered))
    M = len(w_ordered)
    m = np.arange(0, M)
    const = (M - m + 1.5) / M
    if l1_norm == 0.0:
        coeff = np.nan
    else:
        coeff = np.sum((w_ordered / l1_norm) * const)
    return 1.0 - 2.0 * coeff

In [3]:
def chi2_calc(residuals):
    if residuals.dtype == np.complex64 or residuals.dtype == np.complex128:
        data = residuals.real**2 + residuals.imag**2
    else:
        data = residuals**2
    return np.sum(data)

In [4]:
def aicbic(residuals, x):
    rss = chi2_calc(residuals)
    if x.dtype == np.complex64 or x.dtype == np.complex128:
        df = np.count_nonzero(x.real) + np.count_nonzero(x.imag)
    else:
        df = np.count_nonzero(x)
    l = 2 * len(residuals)
    return l * np.log(rss / l) + 2 * df, l * np.log(rss / l) + df * np.log(l)

In [5]:
def list_to2darray(x: list = None, cols: int = None, dtype=None):
    b = list(map(list, zip(*[iter(x)] * cols)))
    #b = [x[cols*i : cols*(i+1)] for i in range(rows)]
    if dtype is None:
        return np.array(b)
    else:
        return np.array(b, dtype=dtype)

In [6]:
class statistics:

    def __init__(self, m, n, z):
        self.sum = np.zeros((m, n), dtype=np.float32)
        self.sum2 = np.zeros((m, n), dtype=np.float32)
        self.n = z * np.ones((m, n), dtype=np.int32)

    def cumul(self, x):
        x_values = np.where(x != np.nan, x, 0.0)
        subtract = np.where(x == np.nan, -1, 0)
        self.sum += x_values
        self.sum2 += x_values * x_values
        self.n += subtract

    def mean(self):
        return np.where(self.n > 0, self.sum / self.n, np.nan)

    def std(self):
        return np.where(
            self.n > 0, np.sqrt(self.sum2 / self.n - self.sum * self.sum / self.n / self.n), np.nan
        )

In [7]:
class Test:

    def __init__(
        self,
        nu_min=None,
        nu_max=None,
        nchannels=None,
        noise_frac=None,
        remove_frac=None,
        use_gridding=False,
        ftransform="nufft",
        use_wavelet=None,
        source_1=None,
        source_2=None,
        scenario=1,
        append_signal=False
    ):
        self.nu_min = nu_min
        self.nu_max = nu_max
        self.nchannels = nchannels
        self.noise_frac = noise_frac
        self.remove_frac = remove_frac
        self.use_gridding = use_gridding
        self.use_wavelet = use_wavelet
        self.ftransform = ftransform
        self.scenario = scenario
        self.append_signal = append_signal
        self.nu = np.linspace(start=nu_min, stop=nu_max, num=nchannels)
        self.source_1 = copy.deepcopy(source_1)
        self.source_2 = copy.deepcopy(source_2)

        if self.source_1 is not None:
            self.source_1.nu = self.nu
            self.source_1.l2_ref = self.source_1.calculate_l2ref()
            self.source_1.simulate()

        if self.source_2 is not None:
            self.source_2.nu = self.nu
            self.source_2.l2_ref = self.source_2.calculate_l2ref()
            self.source_2.simulate()

        if scenario == 1:
            self.source = self.source_1
        elif scenario == 2:
            self.source = self.source_2
        elif scenario == 3:
            self.source = self.source_1 + self.source_2
        else:
            raise ValueError("This scenario does not exist")

        if remove_frac:
            self.source.remove_channels(remove_frac, np.random.RandomState(int(time.time())))

        self.noiseless_source = copy.deepcopy(self.source)

        if scenario == 1:
            self.avg_signal = np.abs(self.source_1.s_nu)
        elif scenario == 2:
            self.avg_signal = np.abs(self.source_2.s_nu)
        else:
            self.avg_signal = (np.abs(self.source_1.s_nu) + np.abs(self.source_2.s_nu)) / 2.0

        if noise_frac:
            self.source.apply_noise(self.avg_signal * self.noise_frac)

        if use_gridding:
            gridding = Gridding(self.source)
            gridding_noiseless = Gridding(self.noiseless_source)
            self.source = gridding.run()
            self.noiseless_source = gridding_noiseless.run()

    def apply_noise(self):
        if self.noise_frac:
            self.source.apply_noise(
                self.avg_signal * self.noise_frac, np.random.RandomState(int(time.time()))
            )

    def run(self, lambda_tv: float = None, lambda_tsv: float = None):
        self.parameter = Parameter()
        self.parameter.calculate_cellsize(dataset=self.source)

        dft = DFT1D(dataset=self.source, parameter=self.parameter)

        self.F_dirty = dft.backward(self.source.data)

        if self.use_wavelet:
            self.wavelet = DiscreteWavelet(
                wavelet_name=self.use_wavelet,
                mode="periodization",
                append_signal=self.append_signal
            )
            #self.wavelet = UndecimatedWavelet(wavelet_name=self.use_wavelet, mode="periodization", append_signal=self.append_signal)
            self.lambda_l1 = np.sqrt(len(self.source.data) + 2 * np.sqrt(len(self.source.data))
                                     ) * 2.0 * np.sqrt(2) * np.mean(self.source.sigma)
        else:
            self.lambda_l1 = np.sqrt(len(self.source.data) + 2 * np.sqrt(len(self.source.data))
                                     ) * np.sqrt(2) * np.mean(self.source.sigma)

        if lambda_tv is None:
            lambda_tv = 0.0

        if lambda_tsv is None:
            lambda_tsv = 0.0

        if self.ftransform == "nufft":
            nufft = NUFFT1D(dataset=self.source, parameter=self.parameter, solve=True)
            if self.use_wavelet:
                chi2 = Chi2(dft_obj=nufft, wavelet=self.wavelet)
            else:
                chi2 = Chi2(dft_obj=nufft)
        else:
            if self.use_wavelet:
                chi2 = Chi2(dft_obj=dft, wavelet=self.wavelet)
            else:
                chi2 = Chi2(dft_obj=dft)

        l1 = L1(reg=self.lambda_l1)
        tsv = TSV(reg=lambda_tsv)
        tv = TV(reg=lambda_tv)
        F_func = [chi2, l1, tsv]
        f_func = [chi2]
        g_func = [l1, tsv]

        F_obj = OFunction(F_func)
        f_obj = OFunction(f_func)
        g_obj = OFunction(g_func)

        self.parameter.data = self.F_dirty

        self.parameter.complex_data_to_real()

        if self.use_wavelet:
            self.parameter.data = self.wavelet.decompose(self.parameter.data)
            opt = FISTA(
                guess_param=self.parameter,
                F_obj=F_obj,
                fx=chi2,
                gx=g_obj,
                noise=2.0 * self.source.theo_noise,
                verbose=False
            )
        else:
            opt = FISTA(
                guess_param=self.parameter,
                F_obj=F_obj,
                fx=chi2,
                gx=g_obj,
                noise=self.source.theo_noise,
                verbose=False
            )

        self.obj, self.X = opt.run()

        if self.use_wavelet is not None:
            self.coeffs = copy.deepcopy(self.X.data)
            k = np.count_nonzero(self.coeffs)
            self.sparsity = k / len(self.coeffs)
            self.gini = gini_coefficient(self.coeffs)
            self.X.data = self.wavelet.reconstruct(self.X.data)
            self.aic, self.bic = aicbic(self.source.residual, self.coeffs)

        else:
            k = np.count_nonzero(self.X.data)
            self.sparsity = k / len(self.X.data)
            self.gini = gini_coefficient(self.X.data)
            self.aic, self.bic = aicbic(self.source.residual, self.X.data)

        self.X.real_data_to_complex()

        self.X_residual = dft.backward(self.source.residual)

        self.X_restored = self.X.convolve() + self.X_residual

        self.res_noise = 0.5 * (np.std(self.X_residual.real) + np.std(self.X_residual.imag))
        self.rmse = np.sqrt(
            np.sum(self.source.residual.real**2 + self.source.residual.imag**2) /
            (2 * len(self.source.residual))
        )
        meaningful_signal = np.where(np.abs(self.parameter.phi) < self.parameter.max_faraday_depth)
        self.signal = np.mean(np.abs(self.X_restored[meaningful_signal]))
        self.peak_signal = np.max(np.abs(self.X_restored))

        self.snr = self.signal / self.res_noise
        self.psnr = self.peak_signal / self.res_noise

        print("Signal-to-noise ratio: {0}".format(self.snr))
        print("Peak Signal-to-noise ratio: {0}".format(self.psnr))
        print("Standard deviation: ({0})*10**5".format(self.res_noise * 10**5))
        print("Root Mean Squared Error: ({0})*10**5".format(self.rmse * 10**5))

        # self.lags, self.autocorr_res, self.autocorr_res_sq, self.bound, self.percentage_real_in, self.percentage_imag_in, self.percentage_real_in_sq, self.percentage_imag_in_sq = self.source.assess_residuals()

        # self.residual_comparison = self.noiseless_source.data - self.source.model_data
        """
        
        self.fig, self.ax = plt.subplots(nrows=2, ncols=4, sharey='row', figsize=(18, 5))

        # Data
        self.ax[0,0].plot(self.source.lambda2, self.source.data.real, 'k.', label=r"Stokes $Q$")
        self.ax[0,0].plot(self.source.lambda2, self.source.data.imag, 'c.', label=r"Stokes $U$")
        self.ax[0,0].plot(self.source.lambda2, np.abs(self.source.data), 'g.', label=r"$|P|$")
        self.ax[0,0].set_xlabel(r'$\lambda^2$[m$^{2}$]')
        self.ax[0,0].set_ylabel(r'Jy/beam')
        self.ax[0,0].title.set_text("Data")

        self.ax[1,0].plot(self.parameter.phi, self.F_dirty.real, 'c--', label=r"Stokes $Q$")
        self.ax[1,0].plot(self.parameter.phi, self.F_dirty.imag, 'c:', label=r"Stokes $U$")
        self.ax[1,0].plot(self.parameter.phi, np.abs(self.F_dirty), 'k-', label=r"|P|")
        self.ax[1,0].set_xlabel(r'$\phi$[rad m$^{-2}$]')
        self.ax[1,0].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
        self.ax[1,0].set_xlim([-1000,1000])

        # Model
        self.ax[0,1].plot(self.source.lambda2, self.source.model_data.real, 'k.', label=r"Stokes $Q$")
        self.ax[0,1].plot(self.source.lambda2, self.source.model_data.imag, 'c.', label=r"Stokes $U$")
        self.ax[0,1].plot(self.source.lambda2, np.abs(self.source.model_data), 'g.', label=r"$|P|$")
        self.ax[0,1].set_xlabel(r'$\lambda^2$[m$^{2}$]')
        self.ax[0,1].set_ylabel(r'Jy/beam')
        self.ax[0,1].title.set_text("Model")

        self.ax[1,1].get_shared_y_axes().remove(self.ax[1,1])
        self.ax[1,1].clear()
        self.ax[1,1].plot(self.parameter.phi, self.X.data.real, 'c--', label=r"Stokes $Q$")
        self.ax[1,1].plot(self.parameter.phi, self.X.data.imag, 'c:', label=r"Stokes $U$")
        self.ax[1,1].plot(self.parameter.phi, np.abs(self.X.data), 'k-', label=r"$|P|$")
        self.ax[1,1].set_xlabel(r'$\phi$[rad m$^{-2}$]')
        self.ax[1,1].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ pix$^{-1}$')
        self.ax[1,1].set_xlim([-1000,1000])
        
        # Residual

        self.ax[0,2].plot(self.source.lambda2, self.source.residual.real, 'k.', label=r"Stokes $Q$")
        self.ax[0,2].plot(self.source.lambda2, self.source.residual.imag, 'c.', label=r"Stokes $U$")
        self.ax[0,2].plot(self.source.lambda2, np.abs(self.source.residual), 'g.', label=r"$|P|$")
        self.ax[0,2].set_xlabel(r'$\lambda^2$[m$^{2}$]')
        self.ax[0,2].set_ylabel(r'Jy/beam')
        self.ax[0,2].title.set_text("Residual")

        self.ax[1,2].plot(self.parameter.phi, self.X_residual.real, 'c--', label=r"Stokes $Q$")
        self.ax[1,2].plot(self.parameter.phi, self.X_residual.imag, 'c:', label=r"Stokes $U$")
        self.ax[1,2].plot(self.parameter.phi, np.abs(self.X_residual), 'k-', label=r"$|P|$")
        self.ax[1,2].set_xlabel(r'$\phi$[rad m$^{-2}$]')
        self.ax[1,2].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
        self.ax[1,2].set_xlim([-1000,1000])
        
        if self.use_wavelet:
            self.ax[0,3].get_shared_y_axes().remove(self.ax[0,3])
            self.ax[0,3].clear()
            self.ax[0,3].plot(self.coeffs)
            self.ax[0,3].title.set_text("Coefficients")

        self.ax[1,3].plot(self.parameter.phi, self.X_restored.real, 'c--', label=r"Stokes $Q$")
        self.ax[1,3].plot(self.parameter.phi, self.X_restored.imag, 'c:', label=r"Stokes $U$")
        self.ax[1,3].plot(self.parameter.phi, np.abs(self.X_restored), 'k-', label=r"$|P|$")
        self.ax[1,3].set_xlim([-1000,1000])
        self.ax[1,3].set_xlabel(r'$\phi$[rad m$^{-2}$]')
        self.ax[1,3].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
        self.ax[1,3].title.set_text("Restored")
        
        self.fig.tight_layout()
        """

In [8]:
def run_test(
    source_1,
    source_2,
    nsigma,
    remove_frac,
    nu_min=1.008e9,
    nu_max=2.031e9,
    nchannels=1000,
    scenario=1,
    use_wavelet=None,
    append_signal=False
):
    nosigma_objs = []
    for remv_frac in remove_frac:
        nosigma_objs.append(
            Test(
                nu_min=nu_min,
                nu_max=nu_max,
                nchannels=nchannels,
                noise_frac=0.0,
                remove_frac=remv_frac,
                scenario=scenario,
                source_1=source_1,
                source_2=source_2,
                use_wavelet=use_wavelet,
                append_signal=append_signal
            )
        )

    test_objs = []
    for nsig in nsigma:
        for i in range(0, len(remove_frac)):
            copy_object = copy.deepcopy(nosigma_objs[i])
            copy_object.noise_frac = nsig
            copy_object.apply_noise()
            test_objs.append(copy_object)

    del nosigma_objs

    nid = len(nsigma) * len(remove_frac)
    for _id in range(0, nid):
        test_objs[_id].run()

    return test_objs

In [9]:
# JVLA 1.008 - 2.031 GHz 546 channels
# MeerKAT 0.9 GHz-1.420 GHz 546
# eMERLIN 1.230 - 1.740 GHz 4096
def run_tests(
    source_1,
    source_2,
    nsigma,
    remove_frac,
    nsamples,
    nu_min=1.008e9,
    nu_max=2.031e9,
    nchannels=1000,
    scenario=1,
    use_wavelet=None,
    append_signal=False
):
    m = len(nsigma)
    n = len(remove_frac)
    psnrs = statistics(m, n, nsamples)
    rmses = statistics(m, n, nsamples)
    #noises = statistics(m, n, nsamples)
    sparsities = statistics(m, n, nsamples)
    #ginies = statistics(m, n, nsamples)
    aics = statistics(m, n, nsamples)
    bics = statistics(m, n, nsamples)

    for i in range(0, nsamples):
        test = run_test(
            source_1,
            source_2,
            nsigma,
            remove_frac,
            nu_min=nu_min,
            nu_max=nu_max,
            nchannels=nchannels,
            scenario=scenario,
            use_wavelet=use_wavelet,
            append_signal=append_signal
        )
        psnrs.cumul(list_to2darray([x.psnr for x in test], n, dtype=np.float32))
        #noises.cumul(list_to2darray([x.res_noise for x in test], n, dtype=np.float32))
        sparsities.cumul(list_to2darray([x.sparsity * 100.0 for x in test], n, dtype=np.float32))
        #ginies.cumul(list_to2darray([x.gini for x in test], n, dtype=np.float32))
        rmses.cumul(list_to2darray([x.rmse for x in test], n, dtype=np.float32))
        aics.cumul(list_to2darray([x.aic for x in test], n, dtype=np.float32))
        bics.cumul(list_to2darray([x.bic for x in test], n, dtype=np.float32))
        for t in test:
            del t
        test = []

    psnr_mean, psnr_std = psnrs.mean(), psnrs.std()
    rmse_mean, rmse_std = rmses.mean(), rmses.std()
    #noise_mean, noise_std = noises.mean(), noises.std()
    sparsity_mean, sparsity_std = sparsities.mean(), sparsities.std()
    #gini_mean, gini_std = ginies.mean(), ginies.std()
    aic_mean, aic_std = aics.mean(), aics.std()
    bic_mean, bic_std = bics.mean(), bics.std()
    return psnr_mean, psnr_std, rmse_mean, rmse_std, aic_mean, aic_std, bic_mean, bic_std, sparsity_mean, sparsity_std

In [10]:
#source_1 = FaradayThinSource(s_nu=0.0035, phi_gal=-200, spectral_idx=1.0)
source_1 = FaradayThinSource(s_nu=0.0035, phi_gal=-400, spectral_idx=0.7)
#source_2 = FaradayThickSource(s_nu=0.0035, phi_fg=140, phi_center=200, spectral_idx=1.0)
source_2 = FaradayThickSource(s_nu=0.0035, phi_fg=95, phi_center=200, spectral_idx=0.7)

In [11]:
nsigma = [0.2]
remove_frac = [0.3]
scenarios = [1, 2, 3]
family_per_scenario = ["db", "db", "db"]
wavelist_per_scenario = [pywt.wavelist(i, kind="discrete") for i in family_per_scenario]

In [12]:
names = ["PSNR", "RMSE", "AIC", "BIC"]
samples = 50
scenario_1_means = np.empty((len(names), len(wavelist_per_scenario[0])), dtype=np.float32)
scenario_1_stds = np.empty((len(names), len(wavelist_per_scenario[0])), dtype=np.float32)

scenario_2_means = np.empty((len(names), len(wavelist_per_scenario[1])), dtype=np.float32)
scenario_2_stds = np.empty((len(names), len(wavelist_per_scenario[1])), dtype=np.float32)

scenario_3_means = np.empty((len(names), len(wavelist_per_scenario[2])), dtype=np.float32)
scenario_3_stds = np.empty((len(names), len(wavelist_per_scenario[2])), dtype=np.float32)

append_signal = True

for i in range(len(scenarios)):
    for j in range(len(wavelist_per_scenario[i])):
        psnr_mean, psnr_std, rmse_mean, rmse_std, aic_mean, aic_std, bic_mean, bic_std, sparsity_mean, sparsity_std = run_tests(
            source_1,
            source_2,
            nsigma,
            remove_frac,
            samples,
            scenario=scenarios[i],
            use_wavelet=wavelist_per_scenario[i][j],
            append_signal=append_signal,
            nu_min=0.9e9,
            nu_max=1.67e9
        )
        if i == 0:
            scenario_1_means[0, j] = psnr_mean
            scenario_1_means[1, j] = rmse_mean
            scenario_1_means[2, j] = aic_mean
            scenario_1_means[3, j] = bic_mean

            scenario_1_stds[0, j] = psnr_std
            scenario_1_stds[1, j] = rmse_std
            scenario_1_stds[2, j] = aic_std
            scenario_1_stds[3, j] = bic_std
        elif i == 1:
            scenario_2_means[0, j] = psnr_mean
            scenario_2_means[1, j] = rmse_mean
            scenario_2_means[2, j] = aic_mean
            scenario_2_means[3, j] = bic_mean

            scenario_2_stds[0, j] = psnr_std
            scenario_2_stds[1, j] = rmse_std
            scenario_2_stds[2, j] = aic_std
            scenario_2_stds[3, j] = bic_std
        else:
            scenario_3_means[0, j] = psnr_mean
            scenario_3_means[1, j] = rmse_mean
            scenario_3_means[2, j] = aic_mean
            scenario_3_means[3, j] = bic_mean

            scenario_3_stds[0, j] = psnr_std
            scenario_3_stds[1, j] = rmse_std
            scenario_3_stds[2, j] = aic_std
            scenario_3_stds[3, j] = bic_std

FWHM of the main peak of the RMTF: 44.658 rad/m^2
Maximum recovered width structure: 97.396 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15607.897
Signal-to-noise ratio: 1.439446180123266
Peak Signal-to-noise ratio: 125.1997198229619
Standard deviation: (2.7296073312754743)*10**5
Root Mean Squared Error: (72.2336744770339)*10**5
FWHM of the main peak of the RMTF: 44.377 rad/m^2
Maximum recovered width structure: 96.051 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15509.872
Signal-to-noise ratio: 1.4396219393564278
Peak Signal-to-noise ratio: 125.02545046137281
Standard deviation: (2.654082163644489)*10**5
Root Mean Squared Error: (69.74232353994017)*10**5
FWHM of the main peak of the RMTF: 44.105 rad/m^2
Maximum recovered width structure: 97.486 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15414.831
Signal-to-noise ratio: 1.4273110329992755
Peak Signal-to-noise ratio: 126.11214519118154
Standard devi

Signal-to-noise ratio: 1.4637952481238719
Peak Signal-to-noise ratio: 137.38651780530554
Standard deviation: (2.545578172430396)*10**5
Root Mean Squared Error: (70.02449688625231)*10**5
FWHM of the main peak of the RMTF: 56.258 rad/m^2
Maximum recovered width structure: 95.160 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 19662.093
Signal-to-noise ratio: 1.471202813531656
Peak Signal-to-noise ratio: 131.66660979699856
Standard deviation: (2.5845472919172607)*10**5
Root Mean Squared Error: (68.84091963062247)*10**5
FWHM of the main peak of the RMTF: 44.552 rad/m^2
Maximum recovered width structure: 95.160 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15570.970
Signal-to-noise ratio: 1.4318022703908149
Peak Signal-to-noise ratio: 128.58322110453508
Standard deviation: (2.6092880943906493)*10**5
Root Mean Squared Error: (72.4739417168584)*10**5
FWHM of the main peak of the RMTF: 44.570 rad/m^2
Maximum recovered width structure: 95.07

Signal-to-noise ratio: 1.4523320657388261
Peak Signal-to-noise ratio: 128.9652188118505
Standard deviation: (2.6767560484586284)*10**5
Root Mean Squared Error: (71.27326402696929)*10**5
FWHM of the main peak of the RMTF: 44.326 rad/m^2
Maximum recovered width structure: 96.319 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15491.800
Signal-to-noise ratio: 1.4277520274651947
Peak Signal-to-noise ratio: 124.8884077727681
Standard deviation: (2.6163499569520354)*10**5
Root Mean Squared Error: (70.50381474515899)*10**5
FWHM of the main peak of the RMTF: 44.105 rad/m^2
Maximum recovered width structure: 97.486 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15414.831
Signal-to-noise ratio: 1.4405413038300312
Peak Signal-to-noise ratio: 128.38485234208164
Standard deviation: (2.627274443511851)*10**5
Root Mean Squared Error: (70.92103879377677)*10**5
FWHM of the main peak of the RMTF: 50.323 rad/m^2
Maximum recovered width structure: 95.60

Signal-to-noise ratio: 1.4503248046341792
Peak Signal-to-noise ratio: 124.18602073970351
Standard deviation: (2.693299757083878)*10**5
Root Mean Squared Error: (71.57461732474422)*10**5
FWHM of the main peak of the RMTF: 44.105 rad/m^2
Maximum recovered width structure: 97.486 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15414.831
Signal-to-noise ratio: 1.4273314039104095
Peak Signal-to-noise ratio: 126.70071096376813
Standard deviation: (2.8003389161312953)*10**5
Root Mean Squared Error: (73.5762022797813)*10**5
FWHM of the main peak of the RMTF: 73.133 rad/m^2
Maximum recovered width structure: 97.486 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 25560.123
Signal-to-noise ratio: 1.4188072943304408
Peak Signal-to-noise ratio: 119.46052876009023
Standard deviation: (2.728635081439279)*10**5
Root Mean Squared Error: (72.50026071781119)*10**5
FWHM of the main peak of the RMTF: 44.969 rad/m^2
Maximum recovered width structure: 93.12

Signal-to-noise ratio: 1.444268713993189
Peak Signal-to-noise ratio: 127.51047520757731
Standard deviation: (2.650367059686687)*10**5
Root Mean Squared Error: (70.94497328314897)*10**5
FWHM of the main peak of the RMTF: 46.731 rad/m^2
Maximum recovered width structure: 97.486 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 16332.528
Signal-to-noise ratio: 1.4492001581784772
Peak Signal-to-noise ratio: 127.20553605898547
Standard deviation: (2.6922683900920674)*10**5
Root Mean Squared Error: (71.9007477380863)*10**5
FWHM of the main peak of the RMTF: 45.510 rad/m^2
Maximum recovered width structure: 97.486 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15905.683
Signal-to-noise ratio: 1.4422051197761065
Peak Signal-to-noise ratio: 126.00873072180907
Standard deviation: (2.7530371880857274)*10**5
Root Mean Squared Error: (69.90097904309538)*10**5
FWHM of the main peak of the RMTF: 44.377 rad/m^2
Maximum recovered width structure: 96.05

...

Signal-to-noise ratio: 1.6571897611486235
Peak Signal-to-noise ratio: 128.40785121272268
Standard deviation: (2.5863635528367013)*10**5
Root Mean Squared Error: (69.03625192777614)*10**5
FWHM of the main peak of the RMTF: 50.952 rad/m^2
Maximum recovered width structure: 97.486 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 17807.801
Signal-to-noise ratio: 1.6026323472088975
Peak Signal-to-noise ratio: 125.87178742424814
Standard deviation: (2.6125657313968986)*10**5
Root Mean Squared Error: (70.89909475595582)*10**5
FWHM of the main peak of the RMTF: 45.953 rad/m^2
Maximum recovered width structure: 93.743 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 16060.650
Signal-to-noise ratio: 1.6103660585996757
Peak Signal-to-noise ratio: 124.69424409405025
Standard deviation: (2.7010657504433766)*10**5
Root Mean Squared Error: (72.02771331899253)*10**5
FWHM of the main peak of the RMTF: 46.061 rad/m^2
Maximum recovered width structure: 97

Signal-to-noise ratio: 1.5912254787731748
Peak Signal-to-noise ratio: 123.1351796289208
Standard deviation: (2.706211671466008)*10**5
Root Mean Squared Error: (72.78640439900165)*10**5
FWHM of the main peak of the RMTF: 45.976 rad/m^2
Maximum recovered width structure: 93.126 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 16068.517
Signal-to-noise ratio: 1.6073902835055964
Peak Signal-to-noise ratio: 120.83064960216842
Standard deviation: (2.8121798095526174)*10**5
Root Mean Squared Error: (72.17282146442881)*10**5
FWHM of the main peak of the RMTF: 51.896 rad/m^2
Maximum recovered width structure: 91.637 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 18137.588
Signal-to-noise ratio: 1.6440515153192532
Peak Signal-to-noise ratio: 127.68986681224268
Standard deviation: (2.6511908799875528)*10**5
Root Mean Squared Error: (69.52225398446775)*10**5
FWHM of the main peak of the RMTF: 44.856 rad/m^2
Maximum recovered width structure: 97.4

Signal-to-noise ratio: 1.6357088640835014
Peak Signal-to-noise ratio: 124.94241634224653
Standard deviation: (2.7407884772401303)*10**5
Root Mean Squared Error: (72.42713289102741)*10**5
FWHM of the main peak of the RMTF: 44.605 rad/m^2
Maximum recovered width structure: 94.894 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15589.562
Signal-to-noise ratio: 1.6196583483689542
Peak Signal-to-noise ratio: 124.90313719698804
Standard deviation: (2.7073765522800386)*10**5
Root Mean Squared Error: (71.11311774648149)*10**5
FWHM of the main peak of the RMTF: 53.530 rad/m^2
Maximum recovered width structure: 97.216 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 18708.735
Signal-to-noise ratio: 1.6174319761925042
Peak Signal-to-noise ratio: 121.03428255799324
Standard deviation: (2.801279515551869)*10**5
Root Mean Squared Error: (71.99661830947859)*10**5
FWHM of the main peak of the RMTF: 44.105 rad/m^2
Maximum recovered width structure: 97.

Signal-to-noise ratio: 1.6230740119246587
Peak Signal-to-noise ratio: 123.32842722834597
Standard deviation: (2.7143876650370657)*10**5
Root Mean Squared Error: (74.02586105029054)*10**5
FWHM of the main peak of the RMTF: 44.172 rad/m^2
Maximum recovered width structure: 97.126 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15438.285
Signal-to-noise ratio: 1.6047918752160795
Peak Signal-to-noise ratio: 123.77574822061513
Standard deviation: (2.8008258595946245)*10**5
Root Mean Squared Error: (72.37422173246581)*10**5
FWHM of the main peak of the RMTF: 45.510 rad/m^2
Maximum recovered width structure: 97.486 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15905.683
Signal-to-noise ratio: 1.6112433790060388
Peak Signal-to-noise ratio: 123.30156343072152
Standard deviation: (2.7431884518591687)*10**5
Root Mean Squared Error: (70.81696279462048)*10**5
FWHM of the main peak of the RMTF: 44.105 rad/m^2
Maximum recovered width structure: 97

Signal-to-noise ratio: 1.6087370846809892
Peak Signal-to-noise ratio: 124.3368263345688
Standard deviation: (2.7267429686617106)*10**5
Root Mean Squared Error: (72.00991582850199)*10**5
FWHM of the main peak of the RMTF: 44.189 rad/m^2
Maximum recovered width structure: 97.036 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15444.180
Signal-to-noise ratio: 1.5840047934493466
Peak Signal-to-noise ratio: 120.42403289035659
Standard deviation: (2.783248964988161)*10**5
Root Mean Squared Error: (73.33197913401537)*10**5
FWHM of the main peak of the RMTF: 44.588 rad/m^2
Maximum recovered width structure: 94.982 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15583.351
Signal-to-noise ratio: 1.6315812961122926
Peak Signal-to-noise ratio: 131.50723709855134
Standard deviation: (2.6215670004603453)*10**5
Root Mean Squared Error: (70.81096471118565)*10**5
FWHM of the main peak of the RMTF: 44.969 rad/m^2
Maximum recovered width structure: 93.1

In [13]:
undecimated = ""
radiotelescope = "meerkat_red"
np.save(undecimated + "wavelet_" + radiotelescope + "_scenario_1_means.npy", scenario_1_means)
np.save(undecimated + "wavelet_" + radiotelescope + "_scenario_1_stds.npy", scenario_1_stds)

np.save(undecimated + "wavelet_" + radiotelescope + "_scenario_2_means.npy", scenario_2_means)
np.save(undecimated + "wavelet_" + radiotelescope + "_scenario_2_stds.npy", scenario_2_stds)

np.save(undecimated + "wavelet_" + radiotelescope + "_scenario_3_means.npy", scenario_3_means)
np.save(undecimated + "wavelet_" + radiotelescope + "_scenario_3_stds.npy", scenario_3_stds)